In [1]:
import numpy as np

with open("data/day15_sample.txt", "r", encoding="UTF-8") as f:
    lines = f.read().split("\n")
    
def manhattan_distance(s, b):
    return abs(s[0] - b[0]) + abs(s[1] - b[1])

def find_four_points(s, md):
    """Input sensor and build all directions based on manhattan distance"""
    
    # col math
    sc_neg = (s[0], s[1] - md)
    sc_pos = (s[0], s[1] + md)
    
    # row math
    sr_neg = (s[0] - md, s[1])
    sr_pos = (s[0] + md, s[1])
    
    return (sc_neg, sc_pos, sr_neg, sr_pos)
    
def flattenArray(l):
    return np.asarray([item for sublist in l for item in sublist])

def build_coverage(sc_neg, sc_pos, sr_neg, sr_pos, md):
    """We can start at sr_neg which is our high point"""
    
    update_vals = []
    
    
    r = sr_neg[0]
    c = sr_neg[1]
    update_vals.append([(r,c)])
    min_c = c
    max_c = c
    
    # each step we increment row by 1, and increment column range outward until we hit the manhattan dist
    while (max_c - min_c + 1) <= (2 * md - 1):
        #Spread cols
        min_c -= 1
        max_c += 1
        r += 1
        
        update_vals.append([(r, c) for c in range(min_c, max_c + 1)])
        
    # now build upwards from low point
    r = sr_pos[0]
    c = sr_pos[1]
    update_vals.append([(r,c)])
    min_c = c
    max_c = c
    # each step we reduce row by 1, and increment column range outward until we hit the manhattan dist
    while (max_c - min_c + 1) <= (2 * md - 2):
        #Spread cols
        min_c -= 1
        max_c += 1
        r -= 1
        update_vals.append([(r, c) for c in range(min_c, max_c + 1)])
        
    
    # return flattened array
    return flattenArray(update_vals)

In [2]:
sensor_dict = {}
for l in lines:
    
    # sensor data
    sx = int(l.split('x=')[1].split(',')[0])
    sy = int(l.split('y=')[1].split(':')[0])
    
    # beacon data
    beacon = l.split('beacon')[1]
    bx = int(beacon.split('x=')[1].split(',')[0])
    by = int(beacon.split('y=')[1].strip())
    
    # store as row, col
    sensor_dict[(sy,sx)] = (by,bx)

In [3]:
# calculate boundaries and offset, which is used mainly
# for indexing / visualization
long_list = []
for k,v in sensor_dict.items():
    long_list.append(k)
    long_list.append(v)
long_list = np.asarray(long_list)

min_r = np.min(long_list[:,0])
max_r = np.max(long_list[:,0])
min_c = np.min(long_list[:,1])
max_c = np.max(long_list[:,1])

print(min_r, max_r)
print(min_c, max_c)
offset = -1 * min_c

0 22
-2 25


In [4]:
# Build our matrix start (not needed but helpful for visualizing solution)
mat = np.zeros((max_r + 1, max_c - min_c + 1), str)
mat[mat == ''] = '.'
for k,v in sensor_dict.items():
    # calculate sensor
    r,c = k
    mat[r, c + offset] = 'S'
    
    # calculate beacon
    r,c = v
    mat[r, c + offset] = 'B'

for row in mat:
    print(''.join(list(row)))

....S.......................
......................S.....
...............S............
................SB..........
............................
............................
............................
..........S.......S.........
............................
............................
....B.......................
..S.........................
............................
............................
..............S.......S.....
B...........................
...........SB...............
................S..........B
....S.......................
............................
............S......S........
............................
.......................B....


In [5]:
# expand grid for visual
def update_grid(mat, coverage, offset):
    for m in coverage:
        if (m[0] < 0) or (m[0] > mat.shape[0] - 1) or (m[1] < 0 - offset) or (m[1] > mat.shape[1] - offset - 1):
            continue
        elif mat[(m[0], m[1] + offset)] in ['B', 'S']:
            continue
        else:
            mat[(m[0], m[1] + offset)] = "#"
    
    return mat

In [6]:
# full sample:
for k,v in sensor_dict.items():

    md = manhattan_distance(k, v)
    a,b,c,d = find_four_points(k, md)
    coverage = build_coverage(a,b,c,d, md)
    mat = update_grid(mat, coverage, offset = 2)

for row in mat:
    print(''.join(list(row))) 

####S#######################
######################S#####
###############S############
################SB##########
###########################.
##########################..
#########################...
.#########S#######S#####....
..#######################...
.#########################..
####B######################.
##S#############.###########
############################
.###########################
.#############S#######S#####
B###########################
###########SB###############
################S##########B
####S######################.
##########################..
############S######S######..
#########################...
.#######..#############B....


In [7]:
len(np.argwhere(mat[10,:] ==  '#')) 

26